In [ ]:
import torch

torch.set_default_dtype(torch.float64)

In [ ]:
def f_sum(x):
    return torch.sum(x**3)

x = torch.tensor([1.0, 2.0])


full_matrix = torch.func.hessian(f_sum)(x)


diagonal = torch.func.vmap(torch.func.grad(torch.func.grad(f_sum)))(x)

print("--- Method A: Full Hessian (2x2) ---")
print(full_matrix)
# Output:
# tensor([[ 6.,  0.],   <-- 6 is the derivative for input 1.0
#         [ 0., 12.]])  <-- 12 is the derivative for input 2.0

print("\n--- Method B: vmap(grad(grad)) (Vector) ---")
print(diagonal)
# Output:
# tensor([ 6., 12.])

print("\n--- Verification ---")
# Does Method B match the diagonal of Method A?
print(torch.allclose(diagonal, torch.diagonal(full_matrix)))
# Output: True

In [ ]:
import sympy as sp
import numpy as np
import torch

# ============================================================
# 1. SymPy: define u(x, y) and its Laplacian
# ============================================================

# Symbols
x_sym, y_sym = sp.symbols('x y')

# Scalar field u(x, y)
u_sym = sp.sin(x_sym) * sp.exp(y_sym) + x_sym**2 * y_sym

# Laplacian Δu = d²u/dx² + d²u/dy²
lap_sym = sp.diff(u_sym, x_sym, 2) + sp.diff(u_sym, y_sym, 2)

# Lambdify for numerical evaluation
u_fn = sp.lambdify((x_sym, y_sym), u_sym, "numpy")
lap_fn = sp.lambdify((x_sym, y_sym), lap_sym, "numpy")


# ============================================================
# 2. PyTorch: same u(x, y) and Laplacian via autograd
# ============================================================

def u_torch(xy: torch.Tensor) -> torch.Tensor:
    """
    Compute u(x, y) = sin(x) * exp(y) + x^2 * y for a batch of points.

    Parameters
    ----------
    xy : torch.Tensor
        Input tensor of shape (N, 2), where each row contains values [x, y].

    Returns
    -------
    torch.Tensor
        Output tensor of shape (N, 1), where each row is u(x, y) evaluated at that point.
    """
    x = xy[:, 0]
    y = xy[:, 1]
    return torch.sin(x) * torch.exp(y) + x**2 * y

def laplacian_autograd(xy):
    """
    Compute the Laplacian Δu of the function u(x, y) via PyTorch autograd.

    The Laplacian is calculated as Δu = sum_i d²u/dx_i² for i over spatial variables.
    This function performs automatic differentiation to compute the second derivatives
    for each spatial coordinate, then sums them to obtain the Laplacian.

    Parameters
    ----------
    xy : torch.Tensor
        Input tensor of shape (N, 2), where N is the number of points (batch size).
        Each row represents the coordinates [x, y] of a point.

    Returns
    -------
    torch.Tensor
        Output tensor of shape (N, 1), where each row contains the Laplacian
        evaluated at the corresponding input point.

    Notes
    -----
    The Laplacian is computed by summing the second partial derivatives
    with respect to all input coordinates for each point in the batch.
    """
    
    # detach() breaks the computational graph and makes xy a leaf node
    # requires_grad_(True) makes it a leaf node and track its gradient
    xy = xy.detach().requires_grad_(True)
    u = u_torch(xy)              # (N, 1)

    # First derivatives wrt x,y
    grads = torch.autograd.grad(
        u.sum(), xy,
        create_graph=True
    )[0]                          # (N, 2)

    lap = 0.0
    for i in range(xy.shape[1]):
        # d²u/dx_i² = d/dx_i (du/dx_i)
        gi = grads[:, i:i+1]      # (N, 1)
        g2 = torch.autograd.grad(
            gi.sum(), xy,
            create_graph=True
        )[0][:, i:i+1]            # (N, 1)
        lap = lap + g2

    return lap                    # (N, 1)

# ============================================================
# 3. Compare SymPy vs PyTorch on random points
# ============================================================

# Use double precision for a better comparison
torch.set_default_dtype(torch.float64)

# Sample N points in R^2
N = 5
xy_torch = torch.randn(N, 2, requires_grad=True)
xy_np = xy_torch.detach().numpy()
xs, ys = xy_np[:, 0], xy_np[:, 1]

# SymPy Laplacian at those points
lap_sym_np = lap_fn(xs, ys)                # shape (N,)
lap_sym_torch = torch.from_numpy(
    np.asarray(lap_sym_np)
).reshape(N, 1)

# PyTorch autograd Laplacian
lap_torch = laplacian_autograd(xy_torch)   # (N, 1)

# ============================================================
# 4. Print results
# ============================================================

print("\nPoints (x, y):")
print(xy_torch)

print("\nLaplacian from SymPy:")
print(lap_sym_torch)

print("\nLaplacian from PyTorch autograd:")
print(lap_torch)

abs_diff = (lap_torch - lap_sym_torch).abs()
print("\nAbsolute differences:")
print(abs_diff)

print("\nMax |difference| =", abs_diff.max().item())


In [ ]:
import torch
from torch import func as F  # PyTorch 2.x

torch.set_default_dtype(torch.float64)

def u_torch(xy):
    # xy: (..., d)
    x = xy[..., 0]
    y = xy[..., 1]
    return torch.sin(x) * torch.exp(y) + x**2 * y  # returns (...,)

def laplacian_exact(xy):
    """
    xy: (N, d) with requires_grad=True
    returns: (N, 1) Laplacian
    """

    def scalar_u(x):
        # x: (d,)
        return u_torch(x.unsqueeze(0))[0]  # scalar

    # Hessian per point: (d, d)
    hess_fn = F.hessian(scalar_u)

    # vmap over batch: xy: (N, d) → hess: (N, d, d)
    H = F.vmap(hess_fn)(xy)

    # trace over last two dims → (N,)
    lap = H.diagonal(dim1=-2, dim2=-1).sum(-1)
    return lap.unsqueeze(-1)

N, d = 5, 2
xy = torch.randn(N, d, requires_grad=True)
lap = laplacian_exact(xy)

lap